### **This notebook is to get the same sample entries (2000 samples and 551 samples) from the album images dataframe as the customer review sample entries to use both modalities feature vecs as input to a simple MLP. Also added in an AUC and LRAP score test**

In [0]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras import applications
import pandas as pd
import numpy as np

from PIL import Image
import requests
from io import BytesIO
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import train_test_split

In [0]:
dfTracks = pd.read_pickle("dfTracks.pkl")

In [0]:
len(dfTracks.genres.unique().tolist())

In [0]:
dfImages = pd.read_pickle("df_images.pkl")
dfImages['imUrl'][1]
dfImages['amazonID'][1]
dfImages


In [0]:
# getting a sample of 551 
dfText551 = pd.read_csv('551_albums_text.csv')
dfText551
entries_to_keep = dfText551.amazonIDs.tolist()
dfImages = dfImages[dfImages['amazonID'].isin(entries_to_keep)]

In [0]:
dfImages = dfImages.sort_values(by=['amazonID'])
dfImages

In [0]:
# getting a sample of 2000
dfText = pd.read_pickle("df2000_babelfied.pkl")
entries_to_keep = dfText.amazonIDs.tolist()
dfImages = dfImages[dfImages['amazonID'].isin(entries_to_keep)]

In [0]:
dfText = dfText.sort_values(by=['amazonIDs'])
dfText

In [0]:
dfImages = dfImages.sort_values(by=['amazonID'])
dfImages

In [0]:
genres = []

#get genre for each album cover through amazon ID and create new target col
for index, row in dfImages.iterrows():
    idAma = (row['amazonID'])
    genre = (dfTracks[dfTracks.amazon_id == idAma]).genres.tolist()[0]
    genres.append(genre)
    
dfImages["genres"] = genres

In [0]:
dfImages

In [0]:
dfText551

In [0]:
def changeToTuple(x):
    stringListGenres = x.split(",")
    stringListGenres = tuple(stringListGenres)
    return(stringListGenres)

In [0]:
import time
from PIL import Image, ImageOps


def convertImage(x):
    
    status_code = 'not done'
    
    # x is the image URL
    while(status_code == 'not done'):
        try:
            response = requests.get(x)
            img = Image.open(BytesIO(response.content))

            #resizing code from jdhao's blog
            desired_size = 300
            old_size = img.size  

            ratio = float(desired_size)/max(old_size)
            new_size = tuple([int(x*ratio) for x in old_size])
            
            # use thumbnail() or resize() method to resize the input image
            # thumbnail is a in-place operation
            # im.thumbnail(new_size, Image.ANTIALIAS)

            img = img.resize(new_size, Image.ANTIALIAS)
            # create a new image and paste the resized on it

            new_img = Image.new("RGB", (desired_size, desired_size))
            new_img.paste(img, ((desired_size-new_size[0])//2,
                                (desired_size-new_size[1])//2))
            
            
            status_code = 'done'
        except requests.exceptions.ConnectionError:
            time.sleep(5)
            continue

    #PIL format to numpy format(h * w * channels)
    numpy_image = image.img_to_array(new_img)
    numpy_image.shape

    #numpy format to batch format - adds extra dim in (batchsize)
    image_batch = np.expand_dims(numpy_image, axis=0)
    return(image_batch)

In [0]:
#X = dfImages["imUrl"]

dfImages["imUrl"] = dfImages["imUrl"].apply(lambda x: convertImage(x))

In [0]:
dfImages.tail()

In [0]:
y = dfImages["genres"]
y = y.apply(lambda x: changeToTuple(x))

In [0]:
X = dfImages["imUrl"]

In [0]:
# pickling the sample of 551 preprocessed images 
X.to_pickle("df551_imgsVectorized_X.pkl")
y.to_pickle("df551_labelsImgs_y.pkl")

In [0]:
y.to_pickle("df2000_labelsImgs_y.pkl")

In [0]:
#read in label dataframe
y = pd.read_pickle("df2000_labelsImgs_y.pkl")

In [0]:
targetList = y.tolist()

In [0]:
X.to_pickle("df2000_imgsVectorized_X.pkl")

In [0]:
#read in vectorized images dataframe
X = pd.read_pickle("df2000_imgsVectorized_X.pkl")

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

targetLabelsBin = mlb.fit_transform(targetList)


In [0]:
targetLabelsBin.shape

In [0]:
res = np.array(X.values.tolist()).squeeze()

# **Building the Model**

In [0]:
X_train, X_test, y_train, y_test = train_test_split(res, targetLabelsBin, test_size=0.25, random_state=42)


print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("y_test shape: " + str(y_test.shape))

In [0]:
#img_height,img_width = 300,300
from keras.layers import Input, Dense
img = Input(shape = (300, 300, 3))


base_model = applications.resnet50.ResNet50(weights = 'imagenet',include_top = False, input_tensor = img, input_shape = None, pooling = 'avg')

In [0]:
processed_image = preprocess_input(image_batch, mode='caffe')

In [0]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.models import Sequential,Model,load_model
num_classes = y_test.shape[1]


final_layer = base_model.layers[-1].output
dense_layer_1 = Dense(128, activation = 'relu')(final_layer)
output_layer = Dense(num_classes, activation = 'sigmoid')(dense_layer_1)
model = Model(input = img, output = output_layer)

In [0]:
# auc function from stackoverflow user tuomastik
# currently not using this ---> error being caused with this function?
import tensorflow as tf
from keras import backend as K

def auc(y_true, y_pred):
    auc = tf.metrics.AUC(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [0]:
from keras.optimizers import SGD, Adam

adam = Adam(lr=0.0001)
model.compile(optimizer = 'adam',metrics = ['accuracy', tf.keras.metrics.AUC(multi_label = True)], loss = 'binary_crossentropy')

In [0]:
#
model.fit(X_train, y_train, epochs = 20, batch_size = 32, validation_data=(X_test, y_test), validation_freq=5, verbose = 1)

In [0]:
model.save("KerasResNetv1.h5")

In [0]:
model.summary()

# **Testing specific samples, scores, investigating outputs**

In [0]:
X_test[23][None]

In [0]:
#import keras
preds = model.predict(X_test)
#y_prob.shape
#y_classes = y_prob.argmax(axis=-1)

preds[preds>=0.5] = 1
preds[preds<0.5] = 0

In [0]:
preds[0]

In [0]:
len(preds)

In [0]:
result = np.where(preds[0] == 1.)

In [0]:
result

In [0]:
y_test[20]

In [0]:
result2 = np.where(y_test[20] == 1)
result2

In [0]:
from sklearn.metrics import label_ranking_average_precision_score

label_ranking_average_precision_score(y_test, preds)



